In [26]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

In [27]:
# Define the base url to concatenate url strings
urlBase = "https://www.motogp.com"

In [154]:
# Returns JSON
def getCircuits(year):
    endpoint = urlBase + "/en/ajax/results/selector/" + str(year)
    req = requests.get(endpoint)
    return req.json()

# DataFrame Cleaning
def dfCircuits(startYear, endYear):
    df_Circuits = pd.DataFrame(columns=['Shortname', 'Title', 'Circuit', 'Year'])
    for i in range(startYear, endYear+1):
        circ = getCircuits(i)
        for key in circ:
            df_Circuits = df_Circuits.append({'Shortname': circ[key]['shortname'], 'Title': circ[key]['title'], 'Circuit': circ[key]['circuit'], 'Year' : str(i)}, ignore_index=True)
    return df_Circuits
 
# Returns HTML | This shows the final ranking
def getDriversTableYear(year):
    endpoint = urlBase + "/en/ajax/results/parse_final_standing/" + str(year)
    req = requests.get(endpoint)
    soup = BeautifulSoup(req.content, 'html.parser')
    return soup

# HTML Cleaning AJAX DriversTable | Returns Dictionary
def htmlCleanDriversTable( year ):
    html = getDriversTableYear(year)
    drivers = html.find_all('td')
    drivers_table = [i.text.split() for i in drivers]
    drivers_table
    counter = 0
    pos = []
    driver_name = []
    motorbike = []
    nationality = []
    points = []
    years = []
    for i in range(0,len(drivers_table)):
        years.append(year)
        if counter == 0:
            counter +=1
            pos.append(drivers_table[i][0])
        elif counter == 1:
            counter +=1
            try:
                  driver_name.append(drivers_table[i][0] + ' ' + drivers_table[i][1])
            except:
                  driver_name.append('NaN')
        elif counter == 2:
            try:
                  motorbike.append(drivers_table[i][0])
            except:
                  motorbike.append('NaN')
            counter +=1
        elif counter == 3:
            try:
                  nationality.append(drivers_table[i][0])
            except:
                  nationality.append('NaN')
            counter +=1
        elif counter == 4:
            try:
                  points.append(drivers_table[i][0])
            except:
                  points.append('NaN')
            counter = 0
    d = {'Position': pos, 'Driver' : driver_name, 'Motorbike' : motorbike, 'Nationality' : nationality, 'Points' : points, 'Year' : year}
    return d
    
# Returns HTML | This gets the final chart by race 
def getWinnersCircuit(circuit, year):
    url = urlBase + "/en/ajax/results/files/" + str(year) + "/" + circuit  + "/500cc/RAC"
    req = requests.get(url)

    return req

def getCategory(year):
    endpoint = urlBase + "en/ajax/results/selector/" + str(year) + "/VEN"
    return endpoint

# Create Dataframes with JSON Inputs
def dataToJson(data):
    return pd.DataFrame(data.json())


In [157]:
header = ['Position', 'Driver', 'Motorbike', 'Nationality', 'Points', 'Year']
df_winners = pd.DataFrame(columns=header)
for i in range(1950, 2020):
    di = htmlCleanDriversTable(i)
    df = pd.DataFrame(di)
    df_winners = pd.concat([df_winners, df])


In [166]:
df_winners.sample()

,Position,Driver,Motorbike,Nationality,Points,Year
22,20,Nello PAGANI,MV,ITA,2,1955


In [167]:
#df_winners.to_csv('../csv_files/motogp_yearly_table.csv')